# A generative model


# Lesson 1 - Load data

The first thing we did was to download Hans Christian Andersen fairytales from Gutenberg.

What do we need to look out for when cleaning the data:

- Punctuation 
- The file consist of multiple stories --> need to divide it into different parts (might be possible to split after headers written in capital letters) --> we will do this in lesson 5
- Capital letters as headers, might want to delete these
- Change everything to lower caser 
- Illustrations, delete them, written as ([Illustration: _His limbs were numbed, his beautiful eyes were closing.-])


We have to decide the input sequences, which is the number of words that the model will train on. This is also the number of words which will be used as the seed text, when using the model to actually generate new sequences. 

In the tutorial 50 words are suggested. We will start with that and then in the fifth lesson we might try to only use sequences inside sentences, though this will minimize the amount of training data. 

Documentation on the following two coding blocks:
https://www.pythonforbeginners.com/files/reading-and-writing-files-in-python
- How to open text

In [14]:
#import packages

from __future__ import print_function
import re
import string
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import load_model
from pickle import load
from random import randint
from keras.preprocessing.sequence import pad_sequences
import os
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Masking

import os
import re
import string
import tensorflow as tf
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from pickle import dump
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
#nltk.download()

In [15]:
#define a function to load the text into memory
def load_text(filename): #we decide that the function is called load_text
    file = open(filename, 'r', encoding = 'utf-8') #open() is used to open/loading a filename, the mode indicates how it should be opened, r = read, w = writing, a = appending 
    text = file.read() # read the file and assign it to the variable text
    file.close() #close the file again
    return text #ends function and specify what output the want, we want text

In [23]:
#set path and working directory
os.chdir("C:\\Users\\au591024\\Desktop\\Data")
path = "C:\\Users\\au591024\\Desktop\\Data"

files = os.listdir(path)

#concatenate text files
filenames = files


with open('adventures.txt', 'w', encoding="utf8") as outfile:
    for fname in filenames:
        with open(fname, encoding="utf8") as infile:
            outfile.write(infile.read())      

In [26]:

# load document
in_filename = 'adventures.txt'
doc = load_text(in_filename)
print(doc[:200])

﻿Just as a little child holds out its hands to catch the sunbeams, to
feel and to grasp what, so its eyes tell it, is actually there, so,
down through the ages, men have stretched out their hands in e


# Lesson 2 - clean data

In [27]:
#removing illustration descriptions
doc = re.sub(r'\[[^)]*\]', '', doc) #using re.sub function and regular expressions 
#[ - an opening bracket 
#[^()]* - zero or more characters other than those defined, that is, any characters other than [ and ]
#\] - a closing bracket



In [28]:
#remove headers
doc = re.sub(r'[A-Z]{2,}', '', doc) #remove capital letters everytime there is more than two 

print(doc[:1000]) #lets look at it 


﻿Just as a little child holds out its hands to catch the sunbeams, to
feel and to grasp what, so its eyes tell it, is actually there, so,
down through the ages, men have stretched out their hands in eager
endeavour to know their God. And because only through the human was
the divine knowable, the old peoples of the earth made gods of their
heroes and not unfrequently endowed these gods with as many of the
vices as of the virtues of their worshippers. As we read the myths of
the East and the West we find ever the same story. That portion of the
ancient Aryan race which poured from the central plain of Asia,
through the rocky defiles of what we now call "The Frontier," to
populate the fertile lowlands of India, had gods who must once have
been wholly heroic, but who came in time to be more degraded than the
most vicious of lustful criminals. And the Greeks, Latins, Teutons,
Celts, and Slavonians, who came of the same mighty Aryan stock, did
even as those with whom they owned a common anc

Documentation for the following code: https://www.geeksforgeeks.org/python-maketrans-translate-functions/

How to use translate together with the helper function maketrans

In [29]:
# turn the document into clean tokens


def clean_doc(doc): #make a function called clean_doc
	# split into tokens by white space
	tokens = doc.split() #get a list with all words (tokens)
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation) #third argument specifies the character we want to delete
	tokens = [w.translate(table) for w in tokens] #use translation mapping from table and loop through all words in token
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()] #isalpha() checks whether the string consists of alphabetic characters only. Replace word with word if it is alphabetical. 
	# make lower case
	tokens = [word.lower() for word in tokens] #replace word in lower case with word in token 
	return tokens

# The following coding blocks are part of lesson 5

In [30]:
#load nltk library
#nltk.download('punkt')

#define tokenizer, use pretrained from the corpus
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

#assign doc to data
data = doc

#divide into sentences
sentence_list = sent_tokenize(data)

#print the number of sentences
print(len(sentence_list))

#check the type, it's a list
type(sentence_list)

#index to see an example
sentence_list[5] #need to remove the punctuation and have in lower case




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\au591024\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
80842


'Originally they\ngave to their gods of their best.'

In [31]:
#check if it works on one sentence in the sentence list
print(clean_doc(sentence_list[3])) #it does

#make an empty list
sentence_clean_list = []

#make a loop which runs through the list of sentences and uses the clean_doc() function
for sentence in sentence_list:
    y = clean_doc(sentence) #save in y and append to empty list
    sentence_clean_list.append(y)
    

#print(sentence_clean_list[:10])

#check to see if the punctuation is removed and if it's in lower case
print(sentence_clean_list[5]) #it is



        

['that', 'portion', 'of', 'the', 'ancient', 'aryan', 'race', 'which', 'poured', 'from', 'the', 'central', 'plain', 'of', 'asia', 'through', 'the', 'rocky', 'defiles', 'of', 'what', 'we', 'now', 'call', 'the', 'frontier', 'to', 'populate', 'the', 'fertile', 'lowlands', 'of', 'india', 'had', 'gods', 'who', 'must', 'once', 'have', 'been', 'wholly', 'heroic', 'but', 'who', 'came', 'in', 'time', 'to', 'be', 'more', 'degraded', 'than', 'the', 'most', 'vicious', 'of', 'lustful', 'criminals']
['originally', 'they', 'gave', 'to', 'their', 'gods', 'of', 'their', 'best']


We need to check the length of the sentences, as we need to use this then either padding or truncating the sentences into sequences. 

In [32]:
#check the length of a random sentence
print(len(sentence_clean_list[3]))

length = []
for sentence in sentence_clean_list:
    l = len(sentence)
    length.append(l)

    
print(max(length))

print(min(length))


58
348
0


In [33]:
#remove sentences less than 1 word long and longer than 130
sentence_new_list1 = [s for s in sentence_clean_list if len(s) >= 1] 
sentence_new_list = [s for s in sentence_new_list1 if len(s) <= 130]


length_new = []
for sentence in sentence_new_list:
    l = len(sentence)
    length_new.append(l)

    
print(max(length_new))

print(min(length_new))


#check how many sentences we have removed
print('Sentences removed after deleting sentences shorter than 1:', len(sentence_clean_list) - len(sentence_new_list1) )
print('Sentences removed after deleting sentences longer than :', len(sentence_new_list1) - len(sentence_new_list)) 



130
1
Sentences removed after deleting sentences shorter than 1: 393
Sentences removed after deleting sentences longer than : 97


In [34]:
# turn into (not separated) sentence, check method on number 5 in the list
line = ' '.join(sentence_new_list[5]) 

print(line) #it works

#create empty list
sequences_sent = []

#create a loop to do it at all the sentences
for sentence in sentence_new_list:
    line = ' '.join(sentence)
    sequences_sent.append(line)

print(sequences_sent[0])

originally they gave to their gods of their best
as a little child holds out its hands to catch the sunbeams to feel and to grasp what so its eyes tell it is actually there so down through the ages men have stretched out their hands in eager endeavour to know their god


In [35]:
# save the sentences to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w', encoding = 'utf-8')
	file.write(data)
	file.close()

In [37]:
# save sentences to file
out_filename = 'clean_text_sentences.txt'
save_doc(sequences_sent, out_filename)

In [38]:
#load the cleaned data with sentences
doc = load_text('clean_text_sentences.txt')

#split the text according to line shift
lines = doc.split('\n')

#check the type
type(lines)#its a list


print(lines[0])

len(lines)

as a little child holds out its hands to catch the sunbeams to feel and to grasp what so its eyes tell it is actually there so down through the ages men have stretched out their hands in eager endeavour to know their god


80352

In [39]:
tokenizer = Tokenizer() #define function Tokenizer() as the variable tokenizer
tokenizer.fit_on_texts(lines) #fit_on_text() tells it what data to train on. We train it on the entire training data, finds all of the unique words in the data and assigns each a unique integer, from 1 to the total number of words
sequences_sent = tokenizer.texts_to_sequences(lines) #convert each sequence from a list of words to a list of integers

#print(sequences_sent[:100]) #all sequences are represented by vectors, consisting of an integer corresponding to each word in the vector

tokenizer.word_index #check out the mapping of integers to words

print(sequences_sent[0])

[18, 5, 49, 265, 4280, 36, 135, 262, 3, 860, 1, 4529, 3, 616, 2, 3, 4401, 51, 27, 135, 168, 156, 11, 31, 2426, 45, 27, 63, 145, 1, 4530, 210, 34, 832, 36, 50, 262, 8, 2387, 12099, 3, 125, 50, 639]


In the model all the input sequences need to be the same length. We therefore need to pad or truncate the different sentences. When you pad the sequences, you add zeros, so that they all become the same length (lenght of the longest sentence. In that case we will have to create an masking input layer which will ignore padded values. This means that padded inputs have no impact on learning. But we will start by padding all the sentences. 

In [40]:
#will padd zeros in front, pre is the default option
padded = pad_sequences(sequences_sent)
print(padded)


len(padded[3]) #all sequences are now 130 long

[[    0     0     0 ...   125    50   639]
 [    0     0     0 ...     4    50 14117]
 [    0     0     0 ...     1   214   347]
 ...
 [    0     0     0 ...     2  1308    82]
 [    0     0     0 ...     1   633   863]
 [    0     0     0 ...     4    13   242]]


130

In [41]:
#check vocabulary size
vocab_size = len(tokenizer.word_index) + 1 #+1 because indexing of arrays starts at zero, and the first word is assigned the integer of 1

vocab_size

28730

In [42]:
# separate into input and output
sequences_sent = array(padded) #create an array with the sequences

X, y = sequences_sent[:,:-1], sequences_sent[:,-1] #separate into input and output sequences
print(X) #input
print(y) #output
y = to_categorical(y, num_classes=vocab_size) #convert to one-hot encoding
seq_length_sent = X.shape[1] #set sequence length to the number of columns in X


[[   0    0    0 ...    3  125   50]
 [   0    0    0 ... 4402    4   50]
 [   0    0    0 ...  188    1  214]
 ...
 [   0    0    0 ... 1260    2 1308]
 [   0    0    0 ...   19    1  633]
 [   0    0    0 ...  255    4   13]]
[  639 14117   347 ...    82   863   242]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]]
1.0
0.0
129


In [57]:
#create a function for generating new sequences 
def generate_seq(model, tokenizer, seq_length_sent, seed_text_sentence, n_words): #n_words is the number of words to generate 
	result = list() #create and empty list
	in_text = seed_text_sentence #assign the see_text to a variable 
	# generate a fixed number of words
	for _ in range(n_words): #range(), first argument is the number of integers to generate, starting from zero
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0] #take the first sequence
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length_sentence, truncating='pre') #make sure that the sequence is 50 words
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word #add the output word to the sequence we are currently working with 
		result.append(out_word) #append the word to the empty list
	return ' '.join(result) #''.join creates a space between each word in the result list and makes a string

# Things to look at in lecture 5 


Pre-Trained Word Embedding. Extend the model to use pre-trained word2vec or GloVe vectors to see if it results in a better model.

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('Data/glove.6B/glove.6B.100d.txt', encoding = 'utf8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Masking(mask_value=0, input_shape=(seq_length_sent,)))
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=seq_length_sent, trainable = False)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
#model.add(Flatten())
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss
# fit model
model.fit(X, y, batch_size=12, epochs=1) #training the model 

In [ ]:
# save the model to file
model.save('model_advanced_full.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_advanced.pkl', 'wb'))

In [ ]:
#loading model
model= load_model('model_advanced_full.h5')
tokenizer = load(open('tokenizer_advanced.pkl', 'rb')) #r for read

In [11]:
#selecting a seed text
seed_text_sentence = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

NameError: name 'lines' is not defined

In [ ]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length_sent, seed_text_sentence, 100)
print(generated)

#saving generated text
out_filename = 'Data/advanced_output.txt'
save_doc(generated, out_filename)

Input sequences


We could process the data so that the model only ever deals with self-contained sentences and pad or truncate the text to meet this requirement for each input sequence. You could explore this as an extension to this tutorial.
- Instead, to keep the example brief, we will let all of the text flow together and train the model to predict the next word across sentences, paragraphs, and even books or chapters in the text.

- The file consist of multiple stories --> need to divide it into different parts (might be possible to split after headers written in capital letters) --> we will do this in lesson 5

- FInd more fairytales, data is not big enough

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.